In [ ]:
# *******************************************************
# Copyright (C) 2020 AMELLAL Oussama <ouss.amellal@gmail.com>
#
# This file is part of 'Yolov3_2_classes_mask' project.
#
# 'Yolov3_2_classes_mask' project can not be copied and/or distributed without the express permission of AMELLAL Oussama
#  *******************************************************

**0) Préparation de la base de données**

In [ ]:
# Ajouter vos images sélectionnées dans un dossier appelé 'imagesmasque' par exemple
# Télécharger la dernière version de  labelImg (v1.8.0 pour windows) : https://tzutalin.github.io/labelImg/ 
# 1. Extraire le fichier .Zip téléchargé
# 2. Cherchez un dossier nommé 'data' ce dossier contient le fichier 'predefined_classes' ouvrez ce fichier pour le modifier.
# supprimez les noms de classes présentes dans le fichier et ajoutez les noms de vos classes dans notre exemple on détecte
# deux classes, ainsi on ajoute deux lignes :
# masque
# pas de masque
# Enregistrez et fermez le fichier.
# 3. Exécuter le fichier 'LabelImg.exe'
# 4. La fenêtre du logiciel LabelImg s'ouvre, Changer le format de sortie des fichiers à YOLO en cliquant sur le bouton  '</> PascalVOC' 
# 5. Cliquez sur 'Open Dir'
# 6. Sélectionnez le dossier où vous avez placé vos images.
# 7. Une image de la base de données va s'afficher, Cliquez sur 'Create RectBox' et sélectionnez avec la sourie les objets ou l'objet présents dans l'images.
# 8. Une petite fenêtre va s'afficher vous donnant le choix de sélectionner la classe de l'objet, sélectionnez la classe et cliquez sur Ok.
# 9. Cliquez sur save et cliquez sur 'Next Image'
# 10. Refaire les étapes 7, 8 et 9 pour tous les images de la base de données.
# 11. Ouvrez le fichier 'imagesmasque' et sélectionnez (ctrl + A) toutes les images et les fichiers '.txt' présents dans le dossier puis un clique droit sur les fichiers sélectionnés 
# et cliquez 'Ajouter à l'archive...' changez le format de fichier à '.Zip' et cliquez Ok, un fichier 'imagesmasque.zip' sera créé. 


# Remarque : vérifiez que pour chaque image un fichier '.txt' a été créé, ce fichier contient 5 valeurs pour chaque objet présent dans l'image : <object-class> <x> <y> <width> <height>
# <object-class> : un nombre entier qui représente le label de la classe, ce nombre est entre 0 et (Nombre de classes-1)
# <x> <y> <width> <height> : des valeurs flottantes relatives à la largeur et à la hauteur de l'image, varient entre 0,0 et 1,0.
# <x> <y> : sont le centre du rectangle, et <width> <height> la largeur et la hauteur du rectangle sélectionné.

# Après la création de la base de données créer un dossier dans votre Google Drive et nommé-le 'yolov3masque' puis ajoutez dans ce dossier le fichier 'imagesmasque.zip'


**1) Connexion à google drive**


In [ ]:
# Vérifier si le GPU NVIDIA est activé
#smi : System Management Interface
#Documentation smi : http://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf
!nvidia-smi

In [ ]:
# Charger vos fichiers présents dans votre Google Drive
# Cette commande vous donnera un lien pour donner l'autorisation à COLab de charger des fichiers présents 
# dans votre compte google Drive, puis vous devrez copier-coller la clé générée.

from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# Pour simplifier les chemins, On utilise un lien symbolique, puis vous pourrez accéder à vos fichiers Google Drive en utilisant simplement le chemin  /mydrive  
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive
 

In [ ]:
%cd /mydrive/yolov3masque 
!ls # lister les fichiers présents dans le dossier 'yolov3masque'

 **2) Cloner Darknet**



In [ ]:
# Cloner daknet dans google colab
!git clone https://github.com/AlexeyAB/darknet

**3) Compiler Darknet avec le GPU**


In [ ]:
# Activer le GPU et OPENCV 
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**4) Configurer le réseau Darknet pour le Training de YOLOv3**



In [ ]:
# Création du fichier de configuration yolov3
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg # Création d'une copie du fichier yolov3.cfg
!ls

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg
# Explications :
# Batch : nombre de données (images) utilisées dans une itération (c'est-à-dire, une mise à jour du gradient) 
# de l'entraînement du modèle. Plus la taille du lot est élevée, plus vous aurez besoin d'espace mémoire.
# Subdivisions = 16 -> Diviser le lot en 16 "mini-lots" donc 64/16 = 4 images par "mini-lots" et ceci est envoyé au GPU pour le traitement.
# Max_batches : Le nombre maximum d'itérations pour entraîner notre réseau de neurones
# classes=80@classes=1 : La configuration par défaut de yolov3 est entraîner à détecter 80 classes, 
# dans notre exemple on détecte une seule classe 'turtle' ainsi on remplace 80 par 1
# filters=255@filters=18 : règle pour calculer le nombre de filtres : filters=(classes + 5)x3 
# Donc, si le nombre de classes = 1, alors filters = 18

In [ ]:
# Create folder on google drive so that we can save there the weights
#!mkdir "yolov3masque"

In [ ]:
!echo -e 'masque\npas de masque' > data/obj.names # le fichier 'obj.names' présent dans le chemin darknet/data va contenir les labels des classes( une seule pour nous 'tortue')
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3masque' > data/obj.data
# L'option -e est utilisée pour activer l'interprétation par Echo d'instances supplémentaires du caractère de nouvelle ligne. 
# Le fichier 'obj.data' va contenir les éléments suivants : 
# classes : le nombre de classes qu'on veut détecter.
# train.txt : le fichier texte 'train.txt' va contenir le chemin de toutes les images de la base de données.
# backup : est l'endroit où vous souhaitez stocker le fichier de poids yolo, ces fichiers seront les résultats de notre apprentissage et qui vont nous permettre de tester notre modèle.

In [ ]:
# Charger les poids darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**5) Extraction des images de la base de données**

In [ ]:
# Les images sont dans le fichier 'imagesmasque.zip' qui est ajouté dans le dossier 'yolov3masque' de Google Drive
!mkdir data/obj # Création d'un dossier 'obj' dans le chemin darknet/data, ce dossier va contenir toutes les images + fichiers '.txt' de la base de données.
!unzip /mydrive/yolov3masque/imagesmasque.zip -d data/obj

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg" )# list de toutes les images de la base de données.
print(images_list)

In [ ]:
# Création du fichier  train.txt 
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!ls

**6) Commencer le training**

In [ ]:
# Commencer le training
!./darknet detector train "/mydrive/yolov3masque/darknet/data/obj.data" "/mydrive/yolov3masque/darknet/cfg/yolov3_training.cfg" "/mydrive/yolov3masque/darknet/darknet53.conv.74" -dont_show
# A la fin du training récupérer les deux fichiers yolov3_training_last.weights et yolov3_training.cfg, ces fichiers sont utilisés pour tester votre modèle.